<a href="https://colab.research.google.com/github/AllanOnyonka-ltsm/Projects/blob/main/Reverse_Engineer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import pandas as pd
import inspect

from typing import Any,Callable

In [3]:
pkl_path='/content/parkinsons_model.pkl'

In [4]:
with open(pkl_path,'rb') as f:
  model=pickle.load(f)

/tmp/ipython-input-3757864762.py:2: UserWarning: [14:15:39] WARNING: /workspace/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  model=pickle.load(f)


In [5]:
print(type(model))

<class 'sklearn.ensemble._voting.VotingClassifier'>


In [6]:
print(type(model).__module__)

sklearn.ensemble._voting


In [8]:
attributes =[attr for attr in dir(model) if not attr.startswith('__')]
print(len(attributes))

57


In [10]:
df=pd.DataFrame(data=attributes,columns=["Attributes"])
df.head(57)

,Attributes
0,_abc_impl
1,_build_request_for_signature
2,_check_feature_names
3,_check_n_features
4,_check_voting
5,_collect_probas
6,_doc_link_module
7,_doc_link_template
8,_doc_link_url_param_generator
9,_estimator_type


In [11]:
prediction_methods=['predict', 'predict_proba', 'predict_log_proba',
                         'decision_function', 'transform', '__call__']

In [14]:
for pred_methods in prediction_methods:
  if hasattr(model,pred_methods):
    print(f'Confirmed method {pred_methods}')

Confirmed method predict
Confirmed method predict_proba
Confirmed method transform


In [17]:
if hasattr(model,'get_params'):
  print('Confirmed method get_params')
  params=model.get_params()
  for key,val in list(params.items())[:5]:
      print(f'{key}:{val}')


Confirmed method get_params
estimators:[('rf', RandomForestClassifier(max_depth=3, min_samples_split=10, n_estimators=50,
                       random_state=42)), ('xgb', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, ...))]
flatten_transform:True
n_jobs:None
verbose:False
vot

High Level Model Summary :


In [20]:
for name,estimator in model.named_estimators_.items():
  if hasattr(estimator,'feature_names_in_'):
    print(name,estimator.feature_names_in_)

In [21]:
hasattr(model, "feature_names_in_"), getattr(model, "feature_names_in_", None)


(False, None)

In [23]:
xgb = model.estimators_[1][1]
hasattr(xgb, "feature_names_in_"), getattr(xgb, "feature_names_in_", None)


TypeError: 'XGBClassifier' object is not subscriptable

In [22]:
rf = model.estimators_[0][1]
hasattr(rf, "feature_names_in_"), getattr(rf, "feature_names_in_", None)


(False, None)

In [35]:
class ModelExplainer:
  def __init__(self,model,feature_names=None) -> None:
    self.model=model
    self.feature_names=feature_names
    self._determine_prediction_method()
  def _determine_prediction_method(self):
    if hasattr(self.model, 'predict_proba'):
            self.predict_fn = self.model.predict_proba
            self.model_type = 'classifier'

    elif hasattr(self.model, 'predict'):
            self.predict_fn = self.model.predict
            self.model_type = 'regressor'

    elif callable(self.model):
            self.predict_fn = self.model
            self.model_type = 'callable'

    else:
            raise ValueError("Cannot determine prediction method")



In [24]:
rf = model.estimators_[0][1]
rf.n_features_in_


14

TypeError: 'XGBClassifier' object is not subscriptable

In [28]:
xgb = model.named_estimators_["xgb"]


In [30]:
attr=[attr for attr in dir(xgb) if "booster" in attr.lower()]
print(attr)

['_Booster', 'booster', 'get_booster']


In [31]:
xgb.get_booster().save_model("xgb_model.json")


In [32]:
import json

In [34]:
with open('xgb_model.json','r')as f:
  xgb_json=json.load(f)
  x =list(xgb_json.keys())
print(x)




['learner', 'version']
